In [ ]:
# 전이학습

In [ ]:
# 케라스를 사용한 전이 학습

In [122]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import sklearn
import matplotlib.pyplot as plt

(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [123]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # 샌들이나 셔츠에 해당하는 인덱스만 True
    y_A = y[~y_5_or_6] # 샌들이나 셔츠가 아닌 항목만 가져오기
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) #셔츠(class 6)입니까?를 맞추는 이진 분류 문제로 정답을 준비
    return ((X[~y_5_or_6], y_A), (X[y_5_or_6], y_B)) 

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]


In [ ]:
print(X_train_A.shape)
print(X_train_B.shape)

(43986, 28, 28)
(200, 28, 28)


In [ ]:
# Model A 훈련 및 저장

In [124]:
tf.random.set_seed(42)
np.random.seed(42)

model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-2),
                metrics=["accuracy"])

history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 12s 9ms/step - loss: 0.3651 - accuracy: 0.8700 - val_loss: 0.2973 - val_accuracy: 0.8939
Epoch 2/20
1375/1375 [==============================] - 7s 5ms/step - loss: 0.2662 - accuracy: 0.9071 - val_loss: 0.2627 - val_accuracy: 0.9071
Epoch 3/20
1375/1375 [==============================] - 8s 6ms/step - loss: 0.2446 - accuracy: 0.9136 - val_loss: 0.2459 - val_accuracy: 0.9106
Epoch 4/20
1375/1375 [==============================] - 8s 6ms/step - loss: 0.2290 - accuracy: 0.9191 - val_loss: 0.2344 - val_accuracy: 0.9200
Epoch 5/20
1375/1375 [==============================] - 8s 6ms/step - loss: 0.2167 - accuracy: 0.9227 - val_loss: 0.2352 - val_accuracy: 0.9158
Epoch 6/20
1375/1375 [==============================] - 8s 6ms/step - loss: 0.2077 - accuracy: 0.9258 - val_loss: 0.2235 - val_accuracy: 0.9220
Epoch 7/20
1375/1375 [==============================] - 8s 6ms/step - loss: 0.1991 - accuracy: 0.9282 - val_loss: 0.2553 - val_accuracy

In [125]:
model_A.save("my_model_A.h5")

In [ ]:
# Model B 훈련

In [126]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300,100,50,50,50):
    model_B.add(keras.layers.Dense(n_hidden, activation="relu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-2),
                # optimizer=keras.optimizers.SGD(),
                metrics=["accuracy"])

history = model_B.fit(X_train_B, y_train_B, epochs=6,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/6
7/7 [==============================] - 2s 63ms/step - loss: 0.6751 - accuracy: 0.7300 - val_loss: 0.6585 - val_accuracy: 0.7424
Epoch 2/6
7/7 [==============================] - 0s 33ms/step - loss: 0.6477 - accuracy: 0.7300 - val_loss: 0.6344 - val_accuracy: 0.7606
Epoch 3/6
7/7 [==============================] - 0s 27ms/step - loss: 0.6206 - accuracy: 0.7750 - val_loss: 0.6075 - val_accuracy: 0.7880
Epoch 4/6
7/7 [==============================] - 0s 28ms/step - loss: 0.5923 - accuracy: 0.8300 - val_loss: 0.5798 - val_accuracy: 0.7931
Epoch 5/6
7/7 [==============================] - 0s 22ms/step - loss: 0.5632 - accuracy: 0.8300 - val_loss: 0.5520 - val_accuracy: 0.8286
Epoch 6/6
7/7 [==============================] - 0s 23ms/step - loss: 0.5326 - accuracy: 0.8750 - val_loss: 0.5220 - val_accuracy: 0.8550


In [127]:
model_B.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 3ms/step - loss: 0.5224 - accuracy: 0.8345


[0.5223622918128967, 0.8345000147819519]

In [128]:
# Model A 가져오기 그리고 복제
model_A = keras.models.load_model("my_model_A.h5")

# 만약 아래와 같이 가져올 경우
# 출력층만 제외하고 모든 층 가져오기
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])

# 새로운 출력층 추가
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))  # 이진 분류

In [ ]:
# Model A의 clone으로 Model_B_A준비

In [129]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())
model_B_on_A = keras.models.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [131]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",
                  optimizer=keras.optimizers.SGD(learning_rate=1e-2),
                     metrics=["accuracy"])

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=10, validation_data=(X_valid_B, y_valid_B))


Epoch 1/10
7/7 [==============================] - 1s 90ms/step - loss: 0.3029 - accuracy: 0.8800 - val_loss: 0.2787 - val_accuracy: 0.8996
Epoch 2/10
7/7 [==============================] - 0s 65ms/step - loss: 0.2136 - accuracy: 0.9350 - val_loss: 0.2173 - val_accuracy: 0.9249
Epoch 3/10
7/7 [==============================] - 0s 39ms/step - loss: 0.1638 - accuracy: 0.9500 - val_loss: 0.1801 - val_accuracy: 0.9381
Epoch 4/10
7/7 [==============================] - 0s 44ms/step - loss: 0.1326 - accuracy: 0.9600 - val_loss: 0.1530 - val_accuracy: 0.9533
Epoch 5/10
7/7 [==============================] - 0s 37ms/step - loss: 0.1097 - accuracy: 0.9750 - val_loss: 0.1338 - val_accuracy: 0.9615
Epoch 6/10
7/7 [==============================] - 0s 36ms/step - loss: 0.0932 - accuracy: 0.9750 - val_loss: 0.1202 - val_accuracy: 0.9655
Epoch 7/10
7/7 [==============================] - 0s 67ms/step - loss: 0.0815 - accuracy: 0.9800 - val_loss: 0.1095 - val_accuracy: 0.9706
Epoch 8/10
7/7 [===========

In [132]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 3ms/step - loss: 0.0841 - accuracy: 0.9785


[0.08405598253011703, 0.9785000085830688]